In [28]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from pydub import AudioSegment

# My audio selection
Vengono selezionati solo audio che hanno durata >= 3 secondi e che hanno file audio presente nel dataset

In [ ]:
from pathlib import Path

# file importato dalla repo di birdnet, contiene tutte le labels del modello
all_species = Path("utils/BirdNET_GLOBAL_6K_V2.4_Labels_en_uk.txt").read_text(encoding="utf-8").splitlines()
all_species[:5]

['Abroscopus albogularis_Rufous-faced Warbler',
 'Abroscopus schisticeps_Black-faced Warbler',
 'Abroscopus superciliaris_Yellow-bellied Warbler',
 'Aburria aburri_Wattled Guan',
 'Acanthagenys rufogularis_Spiny-cheeked Honeyeater']

In [24]:
# mappa ogni nome scientifico al suo nome comune (inglese)
species_dict = {}
for specie in all_species:
    scientific_name, common_name = specie.split("_")
    species_dict[scientific_name] = common_name

In [29]:
bird_tags = scipy.io.loadmat('Bird_tags_Train.mat')["Bird_tags"]
coords = {}
inverted_coords = {}
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
for elem in bird_tags:
    # costruisci col formato <scientific name>_<common name>, come preferisce Birdnet 
    tag = elem[0][0][0][0][0]
    scientific_name = tag.replace("_", " ")
    common_name = species_dict.get(scientific_name, "")
    tag = "_".join([scientific_name, common_name])
    file_name = elem[0][0][0][1][0]
    file_path = os.path.join(audio_path, file_name)
    # calcolo della durata
    bbox = elem[0][0][0][3][:4]
    start_time = math.floor(min(bbox[:, 0]))
    end_time = math.floor(max(bbox[:, 0]))
    duration = end_time - start_time
    # scegli solo file che esistono effettivamente e che abbiano una durata >= di 3s
    if not os.path.exists(file_path) or duration < 3:   # la selection
        continue
    if tag not in coords:
        coords[tag] = []
    if file_name not in inverted_coords:
        inverted_coords[file_name] = []
    coords[tag].append({"file_name": file_name, "bbox": bbox.tolist(), "start_time": start_time, "duration": duration, "scientific_name": scientific_name, "common_name": common_name, "tag": tag})
    inverted_coords[file_name].append({"scientific_name": scientific_name, "common_name": common_name, "tag": tag, "start_time": start_time, "duration": duration})


In [26]:
# generate json file of coords
import json
with open("utils/coords.json", "w") as f:
    json.dump(coords, f)
with open("utils/inverted_coords.json", "w") as f:
    json.dump(inverted_coords, f)

In [27]:
# creates custom species list, se necessario
species_list = coords.keys()
with open("utils/custom_species_list.txt", "w") as file:
    file.write("\n".join(species_list))